# VAE Training

In [1]:
import os, datetime
import numpy as np
import joblib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence

## Collect HTML files

Load the HTML files

In [2]:
files = []

for f in os.listdir(os.path.join(os.getcwd(),'html')):
    fd = open(os.path.join(os.getcwd(),'html',f),'r')
    content = fd.read()
    files.append(content)
    fd.close()

Add a mock file as a concatentation of a previous one

In [3]:
files.append(files[0]*10)

## Preprocess the files

Define the padding and trimming parameters

In [4]:
padding='post'

maxlen=200
truncating='post'

Load the tokenizer

In [5]:
SQLi_tokenizer = joblib.load('ignore_tokenizer_20210318105336141483')

Convert the pages into token sequences:

In [6]:
tokens_list = SQLi_tokenizer.texts_to_sequences(files)

Pad and trim the sequences to fit them into an array:

In [7]:
tokens_array = keras.preprocessing.sequence.pad_sequences(tokens_list, padding=padding, maxlen=maxlen, truncating=truncating)

## Defining VAE

Instantiating a VAE module built following [https://keras.io/examples/generative/vae/](https://keras.io/examples/generative/vae/).

*Sampling layer with reparametrization trick* $z_i\sim\mathcal{N}(\mu_\phi, \sigma_\phi)$

In [8]:
class Sampling(layers.Layer):
    
    def call(self,inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch,dim))
        return z_mean + tf.exp(.5 * z_log_var) * epsilon

*Encoder* $Q_\phi(z \vert x)$

In [9]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(200,))

x = layers.Dense(100,activation='relu')(encoder_inputs)
x = layers.Dense(25,activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean,z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean,z_log_var,z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          20100       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 25)           2525        dense[0][0]                      
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            52          dense_1[0][0]                    
____________________________________________________________________________________________

*Decoder* $P_\theta(x\vert z)$

In [10]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(25,activation='relu')(latent_inputs)
x = layers.Dense(100,activation='relu')(x)
decoder_outputs = layers.Dense(200,activation='relu')(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                75        
_________________________________________________________________
dense_3 (Dense)              (None, 100)               2600      
_________________________________________________________________
dense_4 (Dense)              (None, 200)               20200     
Total params: 22,875
Trainable params: 22,875
Non-trainable params: 0
_________________________________________________________________


*VAE with KL+reconstruction loss*

In [11]:
class VAE(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super(VAE,self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = keras.metrics.Mean(name='reconstruction_loss')
        self.kl_loss_tracker = keras.metrics.Mean(name='kl_loss')
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self,data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)           
            reconstruction = self.decoder(z)
            print(reconstruction)
            reconstruction_loss = keras.losses.mean_squared_error(data,reconstruction)
            #reconstruction_loss = tf.reduce_sum(keras.losses.mean_squared_error(data,reconstruction), axis=1)
            kl_loss = -.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = (tf.reduce_sum(kl_loss,axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss,self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads,self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            'loss': self.total_loss_tracker.result(),
            'reconstruction_loss': self.reconstruction_loss_tracker.result(),
            'kl_loss': self.kl_loss_tracker.result(),
        }

## Training the VAE

Instantiating and training the model

In [12]:
vae = VAE(encoder,decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(tokens_array, epochs=2000, batch_size=3)

Epoch 1/2000
Tensor("decoder/dense_4/Relu:0", shape=(None, 200), dtype=float32)
Tensor("decoder/dense_4/Relu:0", shape=(None, 200), dtype=float32)
2/2 [==============================] - 0s 4ms/step - loss: 3804863.0833 - reconstruction_loss: 20915.4844 - kl_loss: 4303964.0000
Epoch 2/2000
2/2 [==============================] - 0s 4ms/step - loss: 349.1680 - reconstruction_loss: 283.2914 - kl_loss: 46.3449
Epoch 3/2000
2/2 [==============================] - 0s 3ms/step - loss: 363.4911 - reconstruction_loss: 282.7825 - kl_loss: 56.5070
Epoch 4/2000
2/2 [==============================] - 0s 6ms/step - loss: 314.8714 - reconstruction_loss: 282.4054 - kl_loss: 63.9185
Epoch 5/2000
2/2 [==============================] - 0s 2ms/step - loss: 372.1857 - reconstruction_loss: 282.4065 - kl_loss: 64.4420
Epoch 6/2000
2/2 [==============================] - 0s 3ms/step - loss: 368.0111 - reconstruction_loss: 282.5163 - kl_loss: 63.4596
Epoch 7/2000
2/2 [==============================] - 0s 3ms/step

2/2 [==============================] - 0s 3ms/step - loss: 249.6512 - reconstruction_loss: 189.4000 - kl_loss: 49.5331
Epoch 62/2000
2/2 [==============================] - 0s 3ms/step - loss: 217.7703 - reconstruction_loss: 185.3684 - kl_loss: 50.0867
Epoch 63/2000
2/2 [==============================] - 0s 3ms/step - loss: 245.3720 - reconstruction_loss: 182.7419 - kl_loss: 50.2154
Epoch 64/2000
2/2 [==============================] - 0s 3ms/step - loss: 210.0944 - reconstruction_loss: 179.5564 - kl_loss: 49.9695
Epoch 65/2000
2/2 [==============================] - 0s 4ms/step - loss: 207.5168 - reconstruction_loss: 177.2162 - kl_loss: 49.4968
Epoch 66/2000
2/2 [==============================] - 0s 3ms/step - loss: 236.1783 - reconstruction_loss: 175.2306 - kl_loss: 49.0139
Epoch 67/2000
2/2 [==============================] - 0s 3ms/step - loss: 202.5330 - reconstruction_loss: 172.9106 - kl_loss: 48.3035
Epoch 68/2000
2/2 [==============================] - 0s 3ms/step - loss: 230.6323 -

2/2 [==============================] - 0s 3ms/step - loss: 158.4858 - reconstruction_loss: 122.0195 - kl_loss: 24.0527
Epoch 123/2000
2/2 [==============================] - 0s 3ms/step - loss: 156.2263 - reconstruction_loss: 120.7295 - kl_loss: 24.2621
Epoch 124/2000
2/2 [==============================] - 0s 3ms/step - loss: 154.9944 - reconstruction_loss: 119.2270 - kl_loss: 24.3727
Epoch 125/2000
2/2 [==============================] - 0s 3ms/step - loss: 152.4332 - reconstruction_loss: 119.6848 - kl_loss: 24.1771
Epoch 126/2000
2/2 [==============================] - 0s 3ms/step - loss: 149.2647 - reconstruction_loss: 117.7781 - kl_loss: 24.2827
Epoch 127/2000
2/2 [==============================] - 0s 3ms/step - loss: 129.2548 - reconstruction_loss: 119.5246 - kl_loss: 23.9801
Epoch 128/2000
2/2 [==============================] - 0s 3ms/step - loss: 149.1230 - reconstruction_loss: 118.4371 - kl_loss: 23.3484
Epoch 129/2000
2/2 [==============================] - 0s 3ms/step - loss: 151

2/2 [==============================] - 0s 2ms/step - loss: 129.8190 - reconstruction_loss: 109.4140 - kl_loss: 11.6635
Epoch 183/2000
2/2 [==============================] - 0s 2ms/step - loss: 132.6527 - reconstruction_loss: 108.4746 - kl_loss: 11.9831
Epoch 184/2000
2/2 [==============================] - 0s 3ms/step - loss: 108.4200 - reconstruction_loss: 109.5350 - kl_loss: 11.8817
Epoch 185/2000
2/2 [==============================] - 0s 3ms/step - loss: 132.3055 - reconstruction_loss: 108.6214 - kl_loss: 11.5908
Epoch 186/2000
2/2 [==============================] - 0s 3ms/step - loss: 103.3179 - reconstruction_loss: 109.9971 - kl_loss: 10.9155
Epoch 187/2000
2/2 [==============================] - 0s 3ms/step - loss: 125.7669 - reconstruction_loss: 109.2719 - kl_loss: 10.7823
Epoch 188/2000
2/2 [==============================] - 0s 3ms/step - loss: 128.9223 - reconstruction_loss: 111.6383 - kl_loss: 10.5110
Epoch 189/2000
2/2 [==============================] - 0s 3ms/step - loss: 126

2/2 [==============================] - 0s 3ms/step - loss: 118.4012 - reconstruction_loss: 98.9678 - kl_loss: 7.6758
Epoch 244/2000
2/2 [==============================] - ETA: 0s - loss: 125.2958 - reconstruction_loss: 116.4383 - kl_loss: 8.857 - 0s 3ms/step - loss: 112.4392 - reconstruction_loss: 98.1175 - kl_loss: 7.8934
Epoch 245/2000
2/2 [==============================] - 0s 3ms/step - loss: 91.6369 - reconstruction_loss: 96.2749 - kl_loss: 8.2420
Epoch 246/2000
2/2 [==============================] - 0s 3ms/step - loss: 93.7291 - reconstruction_loss: 98.7880 - kl_loss: 8.2476
Epoch 247/2000
2/2 [==============================] - 0s 4ms/step - loss: 115.3701 - reconstruction_loss: 95.0464 - kl_loss: 8.7795
Epoch 248/2000
2/2 [==============================] - 0s 3ms/step - loss: 91.7586 - reconstruction_loss: 95.7480 - kl_loss: 9.3031
Epoch 249/2000
2/2 [==============================] - 0s 4ms/step - loss: 110.9642 - reconstruction_loss: 96.1110 - kl_loss: 8.7339
Epoch 250/2000
2/2

...

2/2 [==============================] - 0s 5ms/step - loss: 58.8672 - reconstruction_loss: 46.2496 - kl_loss: 4.8212
Epoch 1720/2000
2/2 [==============================] - 0s 8ms/step - loss: 58.1175 - reconstruction_loss: 46.2346 - kl_loss: 4.8498
Epoch 1721/2000
2/2 [==============================] - 0s 5ms/step - loss: 59.9447 - reconstruction_loss: 46.9869 - kl_loss: 4.9142
Epoch 1722/2000
2/2 [==============================] - 0s 10ms/step - loss: 58.8228 - reconstruction_loss: 46.0338 - kl_loss: 4.8702
Epoch 1723/2000
2/2 [==============================] - 0s 3ms/step - loss: 38.9982 - reconstruction_loss: 46.2099 - kl_loss: 4.7608
Epoch 1724/2000
2/2 [==============================] - 0s 8ms/step - loss: 59.5356 - reconstruction_loss: 46.9952 - kl_loss: 4.8650
Epoch 1725/2000
2/2 [==============================] - 0s 3ms/step - loss: 57.9728 - reconstruction_loss: 45.8982 - kl_loss: 5.0338
Epoch 1726/2000
2/2 [==============================] - 0s 7ms/step - loss: 39.7901 - recons

2/2 [==============================] - 0s 3ms/step - loss: 58.1444 - reconstruction_loss: 45.3546 - kl_loss: 5.1814
Epoch 1782/2000
2/2 [==============================] - 0s 3ms/step - loss: 39.0663 - reconstruction_loss: 45.8872 - kl_loss: 5.1739
Epoch 1783/2000
2/2 [==============================] - 0s 8ms/step - loss: 38.4414 - reconstruction_loss: 45.5600 - kl_loss: 5.0907
Epoch 1784/2000
2/2 [==============================] - 0s 14ms/step - loss: 58.8431 - reconstruction_loss: 46.8534 - kl_loss: 4.9731
Epoch 1785/2000
2/2 [==============================] - 0s 2ms/step - loss: 58.0970 - reconstruction_loss: 46.1597 - kl_loss: 4.8829
Epoch 1786/2000
2/2 [==============================] - 0s 2ms/step - loss: 58.3861 - reconstruction_loss: 45.6361 - kl_loss: 4.9022
Epoch 1787/2000
2/2 [==============================] - 0s 2ms/step - loss: 58.3703 - reconstruction_loss: 46.4399 - kl_loss: 5.0100
Epoch 1788/2000
2/2 [==============================] - 0s 11ms/step - loss: 58.5832 - recon

2/2 [==============================] - 0s 8ms/step - loss: 65.5528 - reconstruction_loss: 48.4774 - kl_loss: 9.1567
Epoch 1844/2000
2/2 [==============================] - 0s 5ms/step - loss: 46.5469 - reconstruction_loss: 49.8146 - kl_loss: 8.8090
Epoch 1845/2000
2/2 [==============================] - 0s 9ms/step - loss: 44.1596 - reconstruction_loss: 46.9184 - kl_loss: 8.5118
Epoch 1846/2000
2/2 [==============================] - 0s 18ms/step - loss: 42.9265 - reconstruction_loss: 47.1611 - kl_loss: 8.1702
Epoch 1847/2000
2/2 [==============================] - 0s 6ms/step - loss: 63.3298 - reconstruction_loss: 46.4260 - kl_loss: 8.1098
Epoch 1848/2000
2/2 [==============================] - 0s 4ms/step - loss: 61.6483 - reconstruction_loss: 45.7482 - kl_loss: 7.9529
Epoch 1849/2000
2/2 [==============================] - 0s 5ms/step - loss: 41.9441 - reconstruction_loss: 45.1088 - kl_loss: 7.9400
Epoch 1850/2000
2/2 [==============================] - 0s 4ms/step - loss: 41.7550 - recons

2/2 [==============================] - 0s 12ms/step - loss: 58.3452 - reconstruction_loss: 44.5452 - kl_loss: 5.0169
Epoch 1906/2000
2/2 [==============================] - 0s 9ms/step - loss: 59.1516 - reconstruction_loss: 47.7914 - kl_loss: 4.7831
Epoch 1907/2000
2/2 [==============================] - 0s 6ms/step - loss: 42.4316 - reconstruction_loss: 51.8385 - kl_loss: 4.4696
Epoch 1908/2000
2/2 [==============================] - 0s 6ms/step - loss: 57.5019 - reconstruction_loss: 45.4540 - kl_loss: 5.0488
Epoch 1909/2000
2/2 [==============================] - 0s 7ms/step - loss: 65.6081 - reconstruction_loss: 50.5565 - kl_loss: 6.2301
Epoch 1910/2000
2/2 [==============================] - 0s 12ms/step - loss: 79.1593 - reconstruction_loss: 60.0320 - kl_loss: 7.7358
Epoch 1911/2000
2/2 [==============================] - 0s 6ms/step - loss: 43.0644 - reconstruction_loss: 48.0801 - kl_loss: 7.3049
Epoch 1912/2000
2/2 [==============================] - 0s 7ms/step - loss: 45.9828 - recon

2/2 [==============================] - 0s 7ms/step - loss: 66.9349 - reconstruction_loss: 47.8622 - kl_loss: 10.5358
Epoch 1968/2000
2/2 [==============================] - 0s 3ms/step - loss: 43.0968 - reconstruction_loss: 45.8515 - kl_loss: 9.9626
Epoch 1969/2000
2/2 [==============================] - 0s 4ms/step - loss: 42.7263 - reconstruction_loss: 46.4071 - kl_loss: 9.8860
Epoch 1970/2000
2/2 [==============================] - 0s 5ms/step - loss: 64.3026 - reconstruction_loss: 45.9142 - kl_loss: 10.0291
Epoch 1971/2000
2/2 [==============================] - 0s 7ms/step - loss: 43.0237 - reconstruction_loss: 45.4599 - kl_loss: 9.7347
Epoch 1972/2000
2/2 [==============================] - 0s 4ms/step - loss: 63.0499 - reconstruction_loss: 45.0791 - kl_loss: 9.8355
Epoch 1973/2000
2/2 [==============================] - 0s 4ms/step - loss: 63.9531 - reconstruction_loss: 44.9606 - kl_loss: 9.7742
Epoch 1974/2000
2/2 [==============================] - 0s 4ms/step - loss: 41.8406 - recon

## Checking the VAE

Printing the encoding of the first three data points (showing $\mu_\phi, \log\sigma_\phi, z$)

In [13]:
vae.encoder(tokens_array[0:3])

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 1.5895084 , -1.5412844 ],
        [-2.3142931 ,  0.31855693],
        [-1.3365623 , -2.065335  ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-5.1746664, -2.7796502],
        [-2.2312315, -1.8900306],
        [-2.8823164, -1.1418735]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 1.537319  , -1.2586347 ],
        [-1.6780676 , -0.14333054],
        [-1.8338729 , -2.644469  ]], dtype=float32)>]

Computing the encoding and decoding of the first datapoint

In [14]:
xtilde = vae.decoder(vae.encoder(tokens_array[0:1]))

Rounding the computed representation to integers

In [15]:
xtilde_int = np.round(xtilde)

Printing the original and the reconstructed representation.

In [16]:
len(tokens_array[0])

200

In [17]:
tokens_array[0]

array([13, 14, 15, 16, 47,  4,  5, 10, 11, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 48, 12, 49, 12, 30, 31,  4,  5, 10, 11, 32, 33,
        6, 34, 35, 36,  6, 37,  7,  1,  8,  9, 38,  2,  3, 39,  7,  1,  8,
        9, 40,  2,  3, 41,  1, 42,  2, 43,  3, 44, 45, 46,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [18]:
xtilde_int.shape

(1, 200)

In [19]:
xtilde_int

array([[13., 14., 15., 16., 46.,  4.,  5., 13., 11., 17., 18., 19., 20.,
        21., 22., 23., 24., 25., 26., 27., 28., 28., 47., 11., 46., 12.,
        29., 31.,  4.,  6., 11.,  0., 31., 31.,  8., 34., 32., 35.,  6.,
        35.,  7.,  0.,  9.,  9., 36.,  0.,  3., 37.,  7.,  0., 12.,  9.,
        37.,  0.,  0., 39.,  0., 41.,  0., 36.,  0., 41.,  0., 44.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [20]:
SQLi_tokenizer.sequences_to_texts(tokens_array[0:1])

['<!-- saved from url= 0042 http://jabba hackingarena no:806/index php --> <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the flag! <br>some content some content <form action= http://jabba hackingarena no:806/index php method= post > <table width= 100 > <tbody><tr><td>Name:</td> <td><input type= text name= username value= ></td></tr> <tr><td>Password:</td> <td><input type= text name= passwd value= ></td></tr> <tr><td><input type= submit value= Submit ></td></tr> </tbody></table> </form> </body></html>']

In [21]:
SQLi_tokenizer.sequences_to_texts(xtilde_int)

['<!-- saved from url= </body></html> http://jabba hackingarena <!-- php --> <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the the 0042 php </body></html> content flag! action= http://jabba > php action= action= text <table method= width= > width= <td><input name= name= 100 ></td></tr> <tbody><tr><td>Name:</td> <td><input content name= <tbody><tr><td>Name:</td> <tr><td>Password:</td> <tr><td><input 100 <tr><td><input </tbody></table>']

In [22]:
np.mean(xtilde_int==tokens_array[0])

0.825

## Saving the VAE

Saving the model weights

In [23]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
vae.save_weights('ignore_vae_W_'+timestamp)

Loading the model weights

In [24]:
vae1 = VAE(encoder,decoder)
vae1.load_weights('ignore_vae_W_'+timestamp)

Checking the weights

In [25]:
vae.decoder(vae.encoder(tokens_array[0:1])) == vae1.decoder(vae.encoder(tokens_array[0:1]))

<tf.Tensor: shape=(1, 200), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  